In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [2]:
# word_vector_size = w2v_model.wv.vectors.shape[1]
word_vector_size = 300
word_vector_size

300

In [3]:
# This is unfortunately very high memory consumptive

# Test here is not the unlabelled testset, but rather the 0.8 split of the original training set

sentence_train_x = np.load("np_arrays/sentences_train_x_0_8_300.npy")
sentence_train_y = np.load("np_arrays/sentences_train_y_0_8_300.npy")

sentence_test_x = np.load("np_arrays/sentences_test_x_0_2_300.npy")
sentence_test_y = np.load("np_arrays/sentences_test_y_0_2_300.npy")

In [4]:
# Just print how many 0 we have, just to be sure that we have shuffled the dataset
y_ = np.argmax(sentence_train_y, axis=-1)
len(y_[y_==0])

999771

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint
# now perform training on the new features vectors.

# Build a "deep" neural network with 2 hidden layers. When we see that it somehow works,
# we can start doing some cross validation on it.

for layer_size in range(30, 101, 5):
    
    print("\n\nStarting with number of layers: {}".format(layer_size))
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                  )

    filepath="models/weights_{}_layers_of_size_{}_big".format(len(model.layers), layer_size)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(x=sentence_train_x,
              y=sentence_train_y,
              validation_data=(sentence_test_x,  sentence_test_y),
              callbacks=callbacks_list,
              epochs=50,
              use_multiprocessing=True,
              batch_size=128)
        




Starting with number of layers: 30
Instructions for updating:
Colocations handled automatically by placer.


2019-12-16 13:55:04,149 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Train on 2000000 samples, validate on 500000 samples
Instructions for updating:
Use tf.cast instead.


2019-12-16 13:55:04,356 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/50
1998208/2000000 [============================>.] - ETA: 0s - loss: 0.4274 - acc: 0.7938
Epoch 00001: val_acc improved from -inf to 0.80004, saving model to models/weights_3_layers_of_size_30_big
2000000/2000000 [==============================] - 31s 16us/sample - loss: 0.4274 - acc: 0.7938 - val_loss: 0.4162 - val_acc: 0.8000
Epoch 2/50
1996544/2000000 [============================>.] - ETA: 0s - loss: 0.4133 - acc: 0.8020
Epoch 00002: val_acc improved from 0.80004 to 0.80316, saving model to models/weights_3_layers_of_size_30_big
2000000/2000000 [==============================] - 31s 15us/sample - loss: 0.4133 - acc: 0.8020 - val_loss: 0.4105 - val_acc: 0.8032
Epoch 3/50
1996928/2000000 [============================>.] - ETA: 0s - loss: 0.4089 - acc: 0.8045
Epoch 00003: val_acc improved from 0.80316 to 0.80395, saving model to models/weights_3_layers_of_size_30_big
2000000/2000000 [==============================] - 31s 16us/sample - loss: 0.4089 - acc: 0.8045 - val_loss: 0.

...

Epoch 22/50
1997824/2000000 [============================>.] - ETA: 0s - loss: 0.3764 - acc: 0.8225
Epoch 00022: val_acc did not improve from 0.81533
2000000/2000000 [==============================] - 34s 17us/sample - loss: 0.3764 - acc: 0.8225 - val_loss: 0.3903 - val_acc: 0.8147
Epoch 23/50
1997696/2000000 [============================>.] - ETA: 0s - loss: 0.3760 - acc: 0.8226
Epoch 00023: val_acc did not improve from 0.81533
2000000/2000000 [==============================] - 34s 17us/sample - loss: 0.3761 - acc: 0.8226 - val_loss: 0.3947 - val_acc: 0.8125
Epoch 24/50
1998976/2000000 [============================>.] - ETA: 0s - loss: 0.3758 - acc: 0.8227
Epoch 00024: val_acc did not improve from 0.81533
2000000/2000000 [==============================] - 34s 17us/sample - loss: 0.3758 - acc: 0.8227 - val_loss: 0.3939 - val_acc: 0.8139
Epoch 25/50
1998848/2000000 [============================>.] - ETA: 0s - loss: 0.3754 - acc: 0.8230
Epoch 00025: val_acc improved from 0.81533 to 0.815



Starting with number of layers: 95
Train on 2000000 samples, validate on 500000 samples
Epoch 1/50
1997056/2000000 [============================>.] - ETA: 0s - loss: 0.4196 - acc: 0.7978
Epoch 00001: val_acc improved from -inf to 0.80317, saving model to models/weights_3_layers_of_size_95_big
2000000/2000000 [==============================] - 35s 18us/sample - loss: 0.4196 - acc: 0.7978 - val_loss: 0.4095 - val_acc: 0.8032
Epoch 2/50
1999616/2000000 [============================>.] - ETA: 0s - loss: 0.4027 - acc: 0.8075
Epoch 00002: val_acc improved from 0.80317 to 0.80767, saving model to models/weights_3_layers_of_size_95_big
2000000/2000000 [==============================] - 34s 17us/sample - loss: 0.4027 - acc: 0.8075 - val_loss: 0.4030 - val_acc: 0.8077
Epoch 3/50
1999360/2000000 [============================>.] - ETA: 0s - loss: 0.3967 - acc: 0.8108
Epoch 00003: val_acc improved from 0.80767 to 0.81008, saving model to models/weights_3_layers_of_size_95_big
2000000/2000000 [===

1996928/2000000 [============================>.] - ETA: 0s - loss: 0.3731 - acc: 0.8240
Epoch 00027: val_acc did not improve from 0.81523
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3732 - acc: 0.8240 - val_loss: 0.3911 - val_acc: 0.8146
Epoch 28/50
1996928/2000000 [============================>.] - ETA: 0s - loss: 0.3728 - acc: 0.8243
Epoch 00028: val_acc did not improve from 0.81523
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3728 - acc: 0.8243 - val_loss: 0.3914 - val_acc: 0.8144
Epoch 29/50
1998464/2000000 [============================>.] - ETA: 0s - loss: 0.3726 - acc: 0.8243
Epoch 00029: val_acc did not improve from 0.81523
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3726 - acc: 0.8243 - val_loss: 0.3927 - val_acc: 0.8141
Epoch 30/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3724 - acc: 0.8246
Epoch 00030: val_acc did not improve from 0.81523
2000000/20000

Epoch 5/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3891 - acc: 0.8152
Epoch 00005: val_acc improved from 0.81118 to 0.81297, saving model to models/weights_3_layers_of_size_100_big
2000000/2000000 [==============================] - 34s 17us/sample - loss: 0.3891 - acc: 0.8152 - val_loss: 0.3930 - val_acc: 0.8130
Epoch 6/50
1997952/2000000 [============================>.] - ETA: 0s - loss: 0.3870 - acc: 0.8164
Epoch 00006: val_acc improved from 0.81297 to 0.81318, saving model to models/weights_3_layers_of_size_100_big
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3870 - acc: 0.8164 - val_loss: 0.3929 - val_acc: 0.8132
Epoch 7/50
1999360/2000000 [============================>.] - ETA: 0s - loss: 0.3852 - acc: 0.8173
Epoch 00007: val_acc did not improve from 0.81318
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3852 - acc: 0.8173 - val_loss: 0.3940 - val_acc: 0.8122
Epoch 8/50
1998976/2000000 [====

1999744/2000000 [============================>.] - ETA: 0s - loss: 0.3705 - acc: 0.8255
Epoch 00032: val_acc did not improve from 0.81574
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3705 - acc: 0.8254 - val_loss: 0.3946 - val_acc: 0.8150
Epoch 33/50
1997184/2000000 [============================>.] - ETA: 0s - loss: 0.3702 - acc: 0.8255
Epoch 00033: val_acc did not improve from 0.81574
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3702 - acc: 0.8255 - val_loss: 0.3916 - val_acc: 0.8151
Epoch 34/50
1997824/2000000 [============================>.] - ETA: 0s - loss: 0.3699 - acc: 0.8259
Epoch 00034: val_acc did not improve from 0.81574
2000000/2000000 [==============================] - 35s 17us/sample - loss: 0.3699 - acc: 0.8259 - val_loss: 0.3936 - val_acc: 0.8149
Epoch 35/50
1996800/2000000 [============================>.] - ETA: 0s - loss: 0.3698 - acc: 0.8259
Epoch 00035: val_acc did not improve from 0.81574
2000000/20000

## To execute this after having trained the above model

After having decided which one is the best model to use, just load the correct model, set the parameters of the model_star so that it matches the one of the best model, and compute the results!

(The below part needs to be rewritten to take advantage of the python modules)

In [ ]:
# Recreate exactl the best model
model_star = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])

model_star.load_weights('models/weights_2_layers_100_big')

model_star.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
print(model_star)

In [ ]:
# Test it on the test dataset to see if it gives the same best results.
model_star.evaluate(sentence_test_x,  sentence_test_y, verbose=2)[1]
model_star.predict(sentence_test_x)

## GET the test data and predict them

In [5]:
# For non binary models
w2v_model = Word2Vec.load("models/w2v_model_epochs_5_win_5_cbow_300.model")

# For binary models
#w2v_model = gensim.models.KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin', binary=True)

2019-12-16 13:43:05,152 : INFO : loading Word2Vec object from models/w2v_model_epochs_5_win_5_cbow_300.model
2019-12-16 13:43:05,885 : INFO : loading wv recursively from models/w2v_model_epochs_5_win_5_cbow_300.model.wv.* with mmap=None
2019-12-16 13:43:05,885 : INFO : loading vectors from models/w2v_model_epochs_5_win_5_cbow_300.model.wv.vectors.npy with mmap=None
2019-12-16 13:43:06,932 : INFO : setting ignored attribute vectors_norm to None
2019-12-16 13:43:06,933 : INFO : loading vocabulary recursively from models/w2v_model_epochs_5_win_5_cbow_300.model.vocabulary.* with mmap=None
2019-12-16 13:43:06,933 : INFO : loading trainables recursively from models/w2v_model_epochs_5_win_5_cbow_300.model.trainables.* with mmap=None
2019-12-16 13:43:06,934 : INFO : loading syn1neg from models/w2v_model_epochs_5_win_5_cbow_300.model.trainables.syn1neg.npy with mmap=None
2019-12-16 13:43:07,366 : INFO : setting ignored attribute cum_table to None
2019-12-16 13:43:07,367 : INFO : loaded models/w

In [ ]:
test_df_cleaned = pd.read_pickle("dataframes/test_cleaned.pickle")

In [ ]:
test_df_cleaned.head()

In [ ]:
sentence_submission_x = create_sentence_vectors_submission(test_df_cleaned['sentence'], word_vector_size, w2v_model)

In [ ]:
sentence_submission_x.shape

In [ ]:
model_star.predict(sentence_submission_x)

In [ ]:
predictions = []
for el in model_star.predict(sentence_submission_x):
    predictions.append(-1 if el[0] > el[1] else 1)

print(predictions[:10])

results = pd.DataFrame({
    "Id": test_df_cleaned['label'],
    "Prediction": predictions
})

results.head(20)

In [ ]:
results.to_csv('Submission.csv', index=False)